In [1]:
import requests as req
import re
from db import DB

In [2]:
HomePage = 'http://product.pconline.com.cn/mobile'
ENT = 'http://product.pconline.com.cn/mobile/{}s1.shtml'

In [9]:
reItem = re.compile(r'<div class="item-pic">([\s\S]*?)<\/div>')
reInfo = re.compile(r'<a href="(?P<href>.*?)".*?<img.*?alt="(?P<title>[\s\S]*?)"')

In [6]:
res = req.get(ENT.format(25))
html = res.text

In [16]:
def process_model(html):
    model = []
    for item in reItem.findall(html):
        for i in reInfo.finditer(item):
            phone = i.groupdict()
            phone['source'] = 'PConline'
            phone['pid'] = phone.get('href').split("/")[-1].split(".")[0]
            phone['kind'] = 'Overall'
            model.append(phone)
    return model

In [18]:
for page in range(2, 115):
    url = ENT.format((page - 1)*25)
    print(url)
    res = req.get(url)
    models = process_model(res.text)
    
    DB.insert_many(models)

http://product.pconline.com.cn/mobile/25s1.shtml
http://product.pconline.com.cn/mobile/50s1.shtml
http://product.pconline.com.cn/mobile/75s1.shtml
http://product.pconline.com.cn/mobile/100s1.shtml
http://product.pconline.com.cn/mobile/125s1.shtml
http://product.pconline.com.cn/mobile/150s1.shtml
http://product.pconline.com.cn/mobile/175s1.shtml
http://product.pconline.com.cn/mobile/200s1.shtml
http://product.pconline.com.cn/mobile/225s1.shtml
http://product.pconline.com.cn/mobile/250s1.shtml
http://product.pconline.com.cn/mobile/275s1.shtml
http://product.pconline.com.cn/mobile/300s1.shtml
http://product.pconline.com.cn/mobile/325s1.shtml
http://product.pconline.com.cn/mobile/350s1.shtml
http://product.pconline.com.cn/mobile/375s1.shtml
http://product.pconline.com.cn/mobile/400s1.shtml
http://product.pconline.com.cn/mobile/425s1.shtml
http://product.pconline.com.cn/mobile/450s1.shtml
http://product.pconline.com.cn/mobile/475s1.shtml
http://product.pconline.com.cn/mobile/500s1.shtml
htt

In [67]:
#href = 'http://product.pconline.com.cn/mobile/apple/616067_comment.html'
#href = 'http://product.pconline.com.cn/mobile/huawei/637114_comment.html'
href = 'http://product.pconline.com.cn/mobile/samsung/643289_comment.html'

In [68]:
#ent = 'http://pdcmt.pconline.com.cn/front/2015/mtp-list.jsp?productId=616067&filterBy=-1&itemCfgId=-1&order=2&pageNo={}&vId=425065'
#ent = 'http://pdcmt.pconline.com.cn/front/2015/mtp-list.jsp?productId=637114&filterBy=-1&itemCfgId=-1&order=2&pageNo={}&vId=427685'
ent = 'http://pdcmt.pconline.com.cn/front/2015/mtp-list.jsp?productId=643289&filterBy=-1&itemCfgId=-1&order=2&pageNo={}&vId=428104'

In [63]:
def process_comments(html, product, pid):
    reUL = re.compile(r'<ul class="goal-detail">([\s\S]*?)<\/ul>')
    reOverall = re.compile(r'<div class="c-out" score="(.*?)">')
    reFeature = re.compile(r'<li>(?P<feature>[\s\S]*?)：(?P<score>.*?)<\/li>')
    
    comments  = []
    
    for cmt in zip(reUL.findall(html), reOverall.findall(html)):
        score = dict(reFeature.findall(cmt[0]))
        score['overall'] = float(cmt[1])
        score['source'] = 'PConline'
        score['product'] = product
        score['pid'] = pid
        score['kind'] = 'Scores'
        comments.append(score)
    return comments
        

In [56]:
def ent_page(url, pat, product, pageno):
    pid = url.split("/")[-1].split(".")[0]
    res = req.get(url)
    comments = process_comments(res.text, product, pid)
    DB.insert_many(comments)
    
    for page in range(2, pageno + 1):
        url = pat.format(page)
        res = req.get(url)
        comments = process_comments(res.text, product, pid)
        
        DB.insert_many(comments)

In [69]:
ent_page(href, ent, 'S8', 5)

In [58]:
lst = list(DB.find({'kind': 'Overall', 'source': 'PConline'}))

In [66]:
for l in lst:
    if '三星S8' in l.get("title"):
        print(l)

{'_id': ObjectId('59eb55a8b735f761862b59d6'), 'href': 'http://product.pconline.com.cn/mobile/samsung/643289.html', 'title': '三星S8+', 'source': 'PConline', 'pid': '643289', 'kind': 'Overall'}
